![ecommerce_analytics-1224x532](ecommerce_analytics-1224x532.jpg)


As a Data Analyst at an electronics e-commerce company, Voltmart, I have been requested by a peer Machine Learning team to clean the data containing the information about orders made last year. They are planning to further use this cleaned data to build a demand forecasting model. To achieve this, they have shared their requirements regarding the desired output table format.

An analyst shared a parquet file called `"orders_data.parquet"` for me to clean and preprocess. 

I can see the dataset schema below along with the **cleaning requirements**:

## `orders_data.parquet`

| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `order_date` | `timestamp` | Date and time when the order was made | _Modify: Remove orders placed between 12am and 5am (inclusive); convert from timestamp to date_ |
| `time_of_day` | `string` | Period of the day when the order was made | _New column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm_ |
| `order_id` | `long` | Order ID | _N/A_ |
| `product` | `string` | Name of a product ordered | _Remove rows containing "TV" as the company has stopped selling this product; ensure all values are lowercase_ |
| `product_ean` | `double` | Product ID | _N/A_ |
| `category` | `string` | Broader category of a product | _Ensure all values are lowercase_ |
| `purchase_address` | `string` | Address line where the order was made ("House Street, City, State Zipcode") | _N/A_ |
| `purchase_state` | `string` | US State of the purchase address | _New column containing: the State that the purchase was ordered from_ |
| `quantity_ordered` | `long` | Number of product units ordered | _N/A_ |
| `price_each` | `double` | Price of a product unit | _N/A_ |
| `cost_price` | `double` | Cost of production per product unit | _N/A_ |
| `turnover` | `double` | Total amount paid for a product (quantity x price) | _N/A_ |
| `margin` | `double` | Profit made by selling a product (turnover - cost) | _N/A_ |

<br>

In [1]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/16 03:23:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load the orders data from a Parquet file
orders_data = spark.read.parquet('orders_data.parquet')
Convert to Pandas DataFrame and display the first few rows
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


### 1. Filter Orders
- Exclude rows where `order_date` is between 12:00 AM and 5:00 AM.
- Convert `order_date` to a date format.

In [3]:
# Modify: Remove orders placed between 12am and 5am (inclusive); convert from timestamp to date

from pyspark.sql.functions import to_date, hour, col, minute, date_format

# Step_1 Creating new feature - hour
orders_data1 = orders_data.withColumn('time', date_format('order_date', 'HH:mm'))\
                          .filter(~col('time').between('00:00', '05:00'))
orders_data1.limit(24).toPandas()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,21:25
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,14:15
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,13:33
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,20:33
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,11:59
5,2023-01-29 20:22:00,141239,AAA Batteries (4-pack),2.953869e+12,Alimentation,"775 Willow St, San Francisco, CA 94016",1,2.99,1.4950,2.99,1.4950,20:22
6,2023-01-26 12:16:00,141240,27in 4K Gaming Monitor,5.173671e+12,Vêtements,"979 Park St, Los Angeles, CA 90001",1,389.99,128.6967,389.99,261.2933,12:16
7,2023-01-05 12:04:00,141241,USB-C Charging Cable,8.051737e+12,Vêtements,"181 6th St, San Francisco, CA 94016",1,11.95,5.9750,11.95,5.9750,12:04
8,2023-01-01 10:30:00,141242,Bose SoundSport Headphones,1.508418e+12,Électronique,"867 Willow St, Los Angeles, CA 90001",1,99.99,49.9950,99.99,49.9950,10:30
9,2023-01-22 21:20:00,141243,Apple Airpods Headphones,1.386344e+12,Électronique,"657 Johnson St, San Francisco, CA 94016",1,150.00,97.5000,150.00,52.5000,21:20


### 2. Create `time_of_day`
- Categorize orders by time:
  - Morning: 5:00 AM – 12:00 PM.
  - Afternoon: 12:00 PM – 6:00 PM.
  - Evening: 6:00 PM – 12:00 AM.

| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `time_of_day` | `string` | Period of the day when the order was made | _New column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm_ |

In [4]:
orders_data2 = orders_data1.withColumn('time_of_day', 
                                       F.when(col('time').between('05:00', '11:59'), F.lit('morning'))\
                                       .when(col('time').between('12:00', '17:59'), F.lit('afternoon'))\
                                       .when(col('time').between('18:00', '23:59'), F.lit('evening')) 
                                      )
orders_data2.limit(50).toPandas()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time,time_of_day
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,21:25,evening
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,14:15,afternoon
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,13:33,afternoon
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,20:33,evening
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,11:59,morning
5,2023-01-29 20:22:00,141239,AAA Batteries (4-pack),2.953869e+12,Alimentation,"775 Willow St, San Francisco, CA 94016",1,2.99,1.4950,2.99,1.4950,20:22,evening
6,2023-01-26 12:16:00,141240,27in 4K Gaming Monitor,5.173671e+12,Vêtements,"979 Park St, Los Angeles, CA 90001",1,389.99,128.6967,389.99,261.2933,12:16,afternoon
7,2023-01-05 12:04:00,141241,USB-C Charging Cable,8.051737e+12,Vêtements,"181 6th St, San Francisco, CA 94016",1,11.95,5.9750,11.95,5.9750,12:04,afternoon
8,2023-01-01 10:30:00,141242,Bose SoundSport Headphones,1.508418e+12,Électronique,"867 Willow St, Los Angeles, CA 90001",1,99.99,49.9950,99.99,49.9950,10:30,morning
9,2023-01-22 21:20:00,141243,Apple Airpods Headphones,1.386344e+12,Électronique,"657 Johnson St, San Francisco, CA 94016",1,150.00,97.5000,150.00,52.5000,21:20,evening


### 3. Clean Product Data
- Remove rows where the `product` is "TV".
- Convert all product names and categories to lowercase.

| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `product` | `string` | Name of a product ordered | _Remove rows containing "TV" as the company has stopped selling this product; ensure all values are lowercase_ |

In [5]:
from pyspark.sql.functions import col, lower

orders_data3 = orders_data2.filter(~lower(col("product")).rlike("(?i)tv"))
orders_data3.limit(29).toPandas()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time,time_of_day
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,21:25,evening
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,14:15,afternoon
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,13:33,afternoon
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,20:33,evening
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,11:59,morning
5,2023-01-29 20:22:00,141239,AAA Batteries (4-pack),2.953869e+12,Alimentation,"775 Willow St, San Francisco, CA 94016",1,2.99,1.4950,2.99,1.4950,20:22,evening
6,2023-01-26 12:16:00,141240,27in 4K Gaming Monitor,5.173671e+12,Vêtements,"979 Park St, Los Angeles, CA 90001",1,389.99,128.6967,389.99,261.2933,12:16,afternoon
7,2023-01-05 12:04:00,141241,USB-C Charging Cable,8.051737e+12,Vêtements,"181 6th St, San Francisco, CA 94016",1,11.95,5.9750,11.95,5.9750,12:04,afternoon
8,2023-01-01 10:30:00,141242,Bose SoundSport Headphones,1.508418e+12,Électronique,"867 Willow St, Los Angeles, CA 90001",1,99.99,49.9950,99.99,49.9950,10:30,morning
9,2023-01-22 21:20:00,141243,Apple Airpods Headphones,1.386344e+12,Électronique,"657 Johnson St, San Francisco, CA 94016",1,150.00,97.5000,150.00,52.5000,21:20,evening


**Let's validate that we did it correctly.**

In [6]:
orders_data3.where(col('order_id') == 141248).show()

+----------+--------+-------+----------+--------+----------------+----------------+----------+----------+--------+------+----+-----------+
|order_date|order_id|product|product_id|category|purchase_address|quantity_ordered|price_each|cost_price|turnover|margin|time|time_of_day|
+----------+--------+-------+----------+--------+----------------+----------------+----------+----------+--------+------+----+-----------+
+----------+--------+-------+----------+--------+----------------+----------------+----------+----------+--------+------+----+-----------+



### 4. Extract `purchase_state`
- Parse `purchase_address` to create a new column, `purchase_state`, containing the US state.


| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `category` | `string` | Broader category of a product | _Ensure all values are lowercase_ |

In [7]:

orders_data4 = orders_data3.withColumn('category', lower(col('category')))
orders_data4.limit(5).toPandas()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time,time_of_day
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,21:25,evening
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,14:15,afternoon
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,13:33,afternoon
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,20:33,evening
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,11:59,morning


### 5. Ensure Data Consistency
- Validate column data types and handle any missing or invalid entries.

| column | data type | description | cleaning requirements | 
|--------|-----------|-------------|-----------------------|
| `purchase_state` | `string` | US State of the purchase address | _New column containing: the State that the purchase was ordered from_ |

In [8]:
from pyspark.sql.functions import regexp_extract

orders_data5 = orders_data4.withColumn("purchase_state", regexp_extract("purchase_address", r", (\w{2}) \d{5}", 1))
orders_data5.limit(5).toPandas()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time,time_of_day,purchase_state
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,21:25,evening,MA
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,14:15,afternoon,OR
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,13:33,afternoon,CA
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,20:33,evening,CA
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,11:59,morning,TX
